In [2]:
import pandas as pd
df = pd.read_csv('state-wise.csv')

In [3]:
df.head()

,STATE,2012-13,2013-14,2014-15,2015-16,2016-17,2017-18,2018-19,2019-20,2020-21,2021-22
0,Andhra Pradesh,26739,26255,18554,20687,21367,25722,35364,36224,32478,47583
1,Arunanchal Pradesh,1567,2050,1682,3706,2786,3802,6284,4210,6689,6903
2,Assam,4611,5207,6170,4920,8044,9906,14952,16671,26330,23151
3,Bihar,14740,17928,22128,28712,31537,33803,29759,20080,46032,41231
4,Chhattisgarh,7847,6582,7653,8976,10896,11370,10290,17318,14807,18623


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   STATE    32 non-null     object
 1   2012-13  32 non-null     int64 
 2   2013-14  32 non-null     int64 
 3   2014-15  32 non-null     int64 
 4   2015-16  32 non-null     int64 
 5   2016-17  32 non-null     int64 
 6   2017-18  32 non-null     int64 
 7   2018-19  32 non-null     int64 
 8   2019-20  32 non-null     int64 
 9   2020-21  32 non-null     int64 
 10  2021-22  32 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 2.9+ KB


In [5]:
import geopandas as gpd
india_map = gpd.read_file('states_india.shp')

In [6]:
india_map.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cartodb_id  36 non-null     int64   
 1   state_code  36 non-null     int64   
 2   st_nm       36 non-null     object  
 3   geometry    36 non-null     geometry
dtypes: geometry(1), int64(2), object(1)
memory usage: 1.2+ KB


In [7]:
india_map.head()

,cartodb_id,state_code,st_nm,geometry
0,1,0,Telangana,"POLYGON ((78.34088 19.88362, 78.35133 19.88184..."
1,2,35,Andaman & Nicobar Island,"MULTIPOLYGON (((93.71976 7.20707, 93.71909 7.2..."
2,3,28,Andhra Pradesh,"MULTIPOLYGON (((81.39375 16.33380, 81.39224 16..."
3,4,12,Arunanchal Pradesh,"POLYGON ((96.16261 29.38078, 96.16860 29.37432..."
4,5,18,Assam,"MULTIPOLYGON (((89.74323 26.30362, 89.74290 26..."


In [19]:
import pandas as pd
import hvplot.pandas
import panel as pn
import geopandas as gpd
import holoviews as hv

# Load your data (replace with your data file and column names)
df = pd.read_csv('state-wise.csv')
india_map = gpd.read_file('states_india.shp')

# Function to create the line chart
def line_bar(year):
    selected_year_data = df.set_index('STATE').T.loc[str(year)]
    selected_year_data = selected_year_data.iloc[:-1]
    
    line_plot = selected_year_data.hvplot.line(
        xlabel='State', 
        ylabel='Investment (in Rupees)', 
        title=f'Capital Invested in {year}',
        height=400
    ).opts(xrotation=90)

    bar_plot = selected_year_data.hvplot.bar(
        x='STATE', 
        y=year, 
        xlabel='State', 
        ylabel='Investment (in Rupees)', 
        title=f'Capital Invested in {year}',
        height=400,
    ).opts(xrotation=90)
    return line_plot + bar_plot  # Combine line and bar plots

def map(year):
    selected_year_data = df.set_index('STATE').T.loc[str(year)]
    
    # Merge the data with the map based on STATE column in the shapefile
    merged_data = india_map.merge(selected_year_data.to_frame(), left_on='st_nm', right_on = 'STATE', how='left')
    merged_data = merged_data.dropna(subset = year)
    # Create the map plot with hover functionality
    map_plot = merged_data.hvplot(geo=True, hover_cols=["STATE", f'Value_{year}'], 
                                  width=500, height=500, 
                                  cmap='coolwarm', legend=True).opts(tools=['hover'])
    
    return map_plot

# Create the year selector widget
year_selector = pn.widgets.Select(options=list(df.columns[1:]))

# Create line and bar plots using the same year_selector
line_bar_plot = pn.interact(line_bar, year=year_selector)
table_plot = df.set_index('STATE')
map_plot = pn.interact(map, year=year_selector)

template = pn.template.FastGridTemplate(
    title="State-wise Capital Invested By RBI(2012-2022)",
    sidebar=[year_selector],
    main=[
        pn.Row(
            pn.Column(
                pn.pane.Markdown("Line and Bar Plot"),
                line_bar_plot,
            ),
        ),
        pn.Row(
                    pn.pane.Markdown("Map Plot"),
                    map_plot,
                    pn.pane.Markdown("Table Plot"),
                    table_plot,
            ),
    ],
    accent_base_color="#88b8b0",
    header_background="#A01346",
    cols={'lg': 12, 'md': 10, 'sm': 6, 'xs': 4, 'xxs': 2},
    row_height=600,
    sidebar_width=300,
    collapsed_sidebar=True,
    main_max_width='100%',
    theme='default',
)

# Serve the Panel app
template.show()

Launching server at http://localhost:22233


C:\Users\aatiw\OneDrive\Desktop\Capital\lib\site-packages\holoviews\plotting\bokeh\plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
C:\Users\aatiw\OneDrive\Desktop\Capital\lib\site-packages\holoviews\plotting\bokeh\plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(
